# This script shows how to formulate nonlinear bicycle model (5 states, 2 inputs) and find analytical Jacobians (Ak and Bk for TVLQR) with `Symbolics.jl`

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/SSD/Code/Julia/tinympc-julia-old/bicycle_tvlqr/Project.toml`


In [2]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [3]:
# ====================
# Explicit model params (working)
# ====================
@variables x[1:5] u[1:2] 
x = collect(x); u = collect(u)
x0 = [    19.4 ;
    19.5 ;
   0.676 ;
    1.51 ;
   0.175 ]; u0 = [   -1.35 ;
 -0.0615 ]
# Second-order Bicycle Dynamics
function dynamics(x, u)
    a_in = u[1]  # longitudinal acceleration 
    ϕ_in = u[2]  # steering angle rate    

    θ = x[3]  # yaw
    v = x[4]  # steering angle
    δ = x[5]

    c = cos(θ)
    s = sin(θ)

    ẋ = v*c
    ẏ = v*s
    θ̇ = v*tan(δ)
    v̇ = a_in
    δ̇ = ϕ_in  # = u2
    return [ẋ, ẏ, θ̇, v̇, δ̇]
end
function rk4(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function midpoint(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end

sym_jac_A = Symbolics.jacobian(rk4(x, u), x)
sym_jac_B = Symbolics.jacobian(rk4(x, u), u)
# Sub value to validate with ForwardDiff
# sub_dict = Dict()
# [sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
# val_jac_A = substitute.(sym_jac_A, (sub_dict,))
# val_jac_B = substitute.(sym_jac_B, (sub_dict,))
# fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
# fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0)
# display(norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B))
# display(val_jac_A)
# display(val_jac_B)

5×2 Matrix{Num}:
 0.166667(0.01cos(0.05(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) + 0.01cos(0.05tan(x[5])*x[4] + x[3]) + 0.01cos(0.1(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) - 0.0005(0.05u[1] + x[4])*tan(0.05u[2] + x[5])*sin(0.05(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) - 0.0005(0.1u[1] + x[4])*tan(0.05u[2] + x[5])*sin(0.1(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]))  …  0.166667(-0.0005((0.05u[1] + x[4])^2)*(1 + tan(0.05u[2] + x[5])^2)*sin(0.05(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) - 0.0005(1 + tan(0.05u[2] + x[5])^2)*(0.05u[1] + x[4])*(0.1u[1] + x[4])*sin(0.1(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]))
 0.166667(0.01sin(0.05(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) + 0.01sin(0.05tan(x[5])*x[4] + x[3]) + 0.01sin(0.1(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) + 0.0005(0.1u[1] + x[4])*tan(0.05u[2] + x[5])*cos(0.1(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3]) + 0.0005(0.05u[1] + x[4])*cos(0.05(0.05u[1] + x[4])*tan(0.05u[2] + x[5]) + x[3])*tan(0.05u[2] +

In [4]:
# ====================
# Generate C code for jacobians
# ====================
fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianA_Raw, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_GetJacobianB_Raw, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end

In [5]:
# ====================
# Generate C code for dynamics
# ====================
dynamics_rk4_ = rk4(x ,u)
dynamics_rk4 = build_function(dynamics_rk4_, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Dynamics_RK4_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics_rk4.txt","a") do io
        println(io, dynamics_rk4)
end

# This takes reference trajectory data from `.txt` files and print to C files

In [6]:
# Export a vector of vectors to C header and source 
function export_vec_of_vec_to_c(filename, var_type, var_name, data)
    size = length(data)*length(data[1])
    declare = var_type * " " * var_name * "[" * string(size) * "]"
    def = declare * " = {\n"
    for i=1:length(data)
        def = def * "  "
        for j=1:length(data[1])
            def = def * string(data[i][j])
            if j < length(data[1]) 
                def = def * ","
            end
        end
        if i < length(data)
            def = def * ",\n"
        end
    end
    def = def*"}"

    open(filename*".h","a") do io
            println(io, def * ";\n")
    end
    # open(filename*".c","a") do io
    #         println(io, def * ";\n")
    # end
    return true
end
# Export a vector of matrix to C header and source (column-based)
function export_vec_of_mat_to_c(filename, var_type, var_name, data)
    size = length(data)*length(data[1])
    declare = var_type * " " * var_name * "[" * string(size) * "]"
    def = declare * " = {\n"
    for i=1:length(data)
        def = def * "  "
        vdata = vcat(data[i]...)
        for j=1:length(data[1])
            def = def * string(vdata[j])
            if j < length(data[1]) 
                def = def * ","
            end
        end
        if i < length(data)
            def = def * ",\n"
        end
    end
    def = def*"}"

    open(filename*".h","a") do io
            println(io, def * ";\n")
    end
    # open(filename*".c","a") do io
    #         println(io, def * ";\n")
    # end
    return true
end

export_vec_of_mat_to_c (generic function with 1 method)

In [8]:
Nt = 101
nx = 5
nu = 2
using DelimitedFiles
uref1 = readdlm("uref_data.txt")
uref1 = reshape(uref1, nu, Nt-1)
xref1 = readdlm("xref_data.txt")
xref1 = reshape(xref1, nx, Nt)
uref = [uref1[1:nu, i] for i = 1:(Nt-1)]
xref = [xref1[1:nx, i] for i = 1:Nt];

export_vec_of_vec_to_c("al_lqr_ltv_data", "sfloat", "Xref_data", xref)
export_vec_of_vec_to_c("al_lqr_ltv_data", "sfloat", "Uref_data", uref)
# This prints to `al_lqr_ltv_data.h`

true